### sparse Autoencoder for states

In [1]:
import tensorflow as tf
import keras
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import pickle
import math
import copy
from sklearn.model_selection import train_test_split
import pandas as pd

Using TensorFlow backend.


In [2]:
state_features = ['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission']
print (state_features)

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission']


In [3]:
train_df = pd.read_csv('../../data/train_scaled.csv')

In [4]:
train_df.head()

,bloc,icustayid,gender,age,elixhauser,re_admission,SOFA,SIRS,Weight_kg,GCS,...,output_4hourly,cumulated_balance_tev,sedation,mechvent,rrt,died_in_hosp,mortality_90d,reward,vaso_input,iv_input
0,0.000000,12.0,1.0,0.203388,0.000000,0.0,0.304348,0.25,0.183842,1.000000,...,0.000000,0.177479,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0
1,0.222560,12.0,1.0,0.203388,0.000000,0.0,0.130435,0.50,0.183842,1.000000,...,0.782139,0.175772,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0
2,0.356608,12.0,1.0,0.203388,0.000000,0.0,0.086957,0.50,0.183842,1.000000,...,0.767260,0.174328,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0
3,0.452837,12.0,1.0,0.203388,0.000000,0.0,0.217391,0.50,0.183842,1.000000,...,0.000000,0.174328,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0
4,0.000000,14.0,0.0,0.909010,0.142857,0.0,0.217391,0.50,0.204211,0.047619,...,0.657302,0.179999,1.0,1.0,0.0,0.0,1.0,15.0,0.0,4.0


In [5]:
keys = train_df.keys()

In [6]:
test_df = pd.read_csv('../../data/test_scaled.csv')

In [7]:
test_df.head()

,bloc,icustayid,gender,age,elixhauser,re_admission,SOFA,SIRS,Weight_kg,GCS,...,output_4hourly,cumulated_balance_tev,sedation,mechvent,rrt,died_in_hosp,mortality_90d,reward,vaso_input,iv_input
0,0.000000,61.0,0.0,0.3779,0.230769,0.0,0.545455,0.75,0.926956,0.40625,...,0.000000,0.601591,0.0,0.0,0.0,0.0,0.0,15.0,0.0,4.0
1,0.222560,61.0,0.0,0.3779,0.230769,0.0,0.363636,0.75,0.926956,0.37500,...,0.747928,0.607028,0.0,0.0,0.0,0.0,0.0,15.0,0.0,4.0
2,0.356608,61.0,0.0,0.3779,0.230769,0.0,0.363636,1.00,0.926956,0.34375,...,0.713579,0.611279,0.0,0.0,0.0,0.0,0.0,15.0,0.0,4.0
3,0.452837,61.0,0.0,0.3779,0.230769,0.0,0.318182,1.00,0.926956,0.31250,...,0.711891,0.615618,0.0,0.0,0.0,0.0,0.0,15.0,0.0,4.0
4,0.527957,61.0,0.0,0.3779,0.230769,0.0,0.318182,1.00,0.926956,0.28125,...,0.726051,0.619879,0.0,0.0,0.0,0.0,0.0,15.0,0.0,4.0


In [8]:
X_train, X_val, _, _ = train_test_split(train_df, train_df, test_size=0.2, random_state=42)

In [9]:
training_df = pd.DataFrame(X_train, columns=keys)
validating_df = pd.DataFrame(X_val, columns=keys)

In [10]:
training_df.sample(2).loc[:,state_features].head()

,Albumin,Arterial_BE,Arterial_lactate,Arterial_pH,BUN,CO2_mEqL,Calcium,Chloride,Creatinine,DiaBP,...,Weight_kg,age,elixhauser,gender,mechvent,output_4hourly,output_total,paCO2,paO2,re_admission
91646,0.336170,0.379562,0.093291,0.601053,0.400233,0.192857,0.397906,0.607895,0.223964,0.536000,...,0.216158,0.582962,0.142857,1.0,0.0,0.741095,0.764698,0.131932,0.106331,1.0
63425,0.624928,0.462774,0.065095,0.676842,0.360399,0.239796,0.424084,0.607895,0.223964,0.642353,...,0.179174,0.532249,0.357143,1.0,1.0,0.822710,0.648245,0.154699,0.324173,0.0


In [11]:
n_input = len(state_features)
n_hidden_1 = 200
weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.zeros([n_hidden_1])),
    'decoder_b1': tf.Variable(tf.zeros([n_input])),
}

In [12]:
class AutoEncoder:
    def __init__(self):
        self.input_size = n_input
        self.batch_size = tf.placeholder(tf.int32)
        self.sparsity_dist = tf.fill([self.batch_size, n_hidden_1], 0.05)
        self.X = tf.placeholder("float", [None, self.input_size])
        self.layer_1_a = tf.nn.sigmoid(tf.add(tf.matmul(self.X, weights['encoder_h1']),
                                   biases['encoder_b1']))
        self.layer_1_d = tf.nn.sigmoid(tf.add(tf.matmul(self.layer_1_a, weights['decoder_h1']),
                                   biases['decoder_b1']))
        self.target_y = self.X
        self.kl = tf.reduce_mean(tf.reduce_sum(tf.multiply(self.sparsity_dist, tf.log(self.sparsity_dist/self.layer_1_a)) + \
                                   tf.multiply((1-self.sparsity_dist),tf.log((1-self.sparsity_dist)/(1-self.layer_1_a))), axis =1))
        #self.kl = tf.reduce_mean(tf.contrib.keras.losses.kullback_leibler_divergence(self.sparsity_dist, self.layer_1_a))  
    # regularisation term logic - loss at the start is roughly 60, of this 0.5 is reconstruction loss.  
        self.loss = tf.reduce_mean(tf.pow(self.target_y - self.layer_1_d, 2)) + 0.0001 * self.kl
        #tf.summary.scalar('loss', self.loss)
        #self.accuracy = tf.reduce_mean(tf.metrics.accuracy(self.layer_1_d, self.target_y))
        #tf.summary.scalar('acc', self.acc)
        #self.loss = tf.reduce_mean(tf.pow(self.target_y - self.layer_1_d, 2))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(self.loss)

In [13]:
def eval_on_val():
    x = validating_df.loc[:,state_features]
    l = sess.run([autoencoder.loss], feed_dict={autoencoder.X:x, autoencoder.batch_size:X_val.shape[0]})
    print ("validation loss is: ", l)

In [14]:
batch_size = 100
autoencoder = AutoEncoder()
#tf.summary.scalar('loss', autoencoder.loss)
# tf.summary.scalar('acc', autoencoder.acc)
#merged = tf.summary.merge_all()
saver = tf.train.Saver()

init = tf.global_variables_initializer()

with tf.Session() as sess:
    #writer = tf.summary.FileWriter('logs/sparse_auto', sess.graph)
    sess.run(init)
    # Training cycle
    for step in range(100000):
        #sum_loss = 0
        a = training_df.sample(n=batch_size)
        cur_state = a.loc[:, state_features]
        _, l = sess.run([autoencoder.optimizer, autoencoder.loss], feed_dict={autoencoder.X:cur_state, autoencoder.batch_size : batch_size})
        #sum_loss += l
        #writer.add_summary(summary, step)
        if step % 1000==0 and step > 0:
            print ("step is ", step )
            #print ("total loss is ", sum_loss)
            print ("total loss is ", l)
            sum_loss = 0
        if step % 20000 == 0 and step >0:
            b = training_df.sample(n=1)
            cur_state_trial = b.loc[:, state_features]
            target_y, reconstruction = sess.run([autoencoder.target_y,autoencoder.layer_1_d],feed_dict={autoencoder.X:cur_state_trial})
            print (reconstruction)
            print (target_y)
        if step % 20000 == 0 and step >0:
            eval_on_val()
    #save_path = saver.save(sess, "sparse_autoencoder_model/model.ckpt")
    #print ('model saved to ', save_path)
    train_states = train_df.loc[:,state_features]
    train_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:train_states})

    train_autoencode = pd.DataFrame(train_repr[0])
    train_autoencode['vaso_input'] = train_df['vaso_input']
    train_autoencode['iv_input'] = train_df['iv_input']
    train_autoencode['reward'] = train_df['reward']
    train_autoencode['icustayid'] = train_df['icustayid']

    test_states = test_df.loc[:,state_features]
    test_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:test_states})
    test_autoencode = pd.DataFrame(test_repr[0])
    test_autoencode['vaso_input'] = test_df['vaso_input']
    test_autoencode['iv_input'] = test_df['iv_input']
    test_autoencode['reward'] = test_df['reward']
    test_autoencode['icustayid'] = test_df['icustayid']

step is  1000
total loss is  0.092907
step is  2000
total loss is  0.0700562
step is  3000
total loss is  0.0558024
step is  4000
total loss is  0.0469554
step is  5000
total loss is  0.0392559
step is  6000
total loss is  0.0310955
step is  7000
total loss is  0.0270705
step is  8000
total loss is  0.0225123
step is  9000
total loss is  0.0195399
step is  10000
total loss is  0.0166736
step is  11000
total loss is  0.0149521
step is  12000
total loss is  0.0125755
step is  13000
total loss is  0.0121986
step is  14000
total loss is  0.0104676
step is  15000
total loss is  0.00856115
step is  16000
total loss is  0.0072482
step is  17000
total loss is  0.00773185
step is  18000
total loss is  0.0069851
step is  19000
total loss is  0.00533257
step is  20000
total loss is  0.00513382
[[ 0.55763751  0.35801414  0.17168963  0.60590547  0.63653612  0.23511882
   0.39851338  0.54589075  0.31332004  0.50625432  0.23070888  0.91541302
   0.16404289  0.20515084  0.37928942  0.44377631  0.42485

step is  85000
total loss is  0.000476545
step is  86000
total loss is  0.00047271
step is  87000
total loss is  0.000467588
step is  88000
total loss is  0.000476055
step is  89000
total loss is  0.000455426
step is  90000
total loss is  0.000435803
step is  91000
total loss is  0.000499579
step is  92000
total loss is  0.000368181
step is  93000
total loss is  0.000484706
step is  94000
total loss is  0.000404658
step is  95000
total loss is  0.000447708
step is  96000
total loss is  0.000454015
step is  97000
total loss is  0.000441426
step is  98000
total loss is  0.000443749
step is  99000
total loss is  0.000433807


In [15]:
train_autoencode.to_csv("../../data/train_scaled_encoded.csv", index=False)
test_autoencode.to_csv("../../data/test_scaled_encoded.csv", index=False)

### Swap vaso and iv input, fixing glitch

In [16]:
train_encoded = pd.read_csv('../../data/train_scaled_encoded.csv')
test_encoded = pd.read_csv('../../data/test_scaled_encoded.csv')

In [17]:
temp = train_encoded['vaso_input']
train_encoded['vaso_input'] = train_encoded['iv_input']
train_encoded['iv_input'] = temp

In [18]:
train_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,vaso_input,iv_input,reward,icustayid
0,0.064802,0.044244,0.037107,0.059904,0.046192,0.053970,0.082275,0.050313,0.116449,0.043997,...,0.040003,0.117637,0.061369,0.051312,0.121394,0.047884,0.0,0.0,15.0,12.0
1,0.063519,0.067414,0.046285,0.060904,0.046157,0.045159,0.053256,0.067262,0.086092,0.074481,...,0.054532,0.083738,0.069601,0.077168,0.084418,0.048673,0.0,0.0,15.0,12.0
2,0.054806,0.066314,0.046150,0.064969,0.043769,0.042330,0.053180,0.061933,0.094193,0.067433,...,0.052888,0.072617,0.077050,0.076517,0.071960,0.049371,0.0,0.0,15.0,12.0
3,0.044274,0.047118,0.041674,0.049126,0.029567,0.063013,0.080330,0.076667,0.143153,0.031403,...,0.047660,0.040665,0.105482,0.089489,0.093178,0.060669,0.0,0.0,15.0,12.0
4,0.032231,0.049927,0.077559,0.049616,0.057226,0.116102,0.064314,0.028499,0.097456,0.060871,...,0.067804,0.053272,0.116516,0.057016,0.024232,0.065007,4.0,4.0,15.0,14.0


In [19]:
temp = test_encoded['vaso_input']
test_encoded['vaso_input'] = test_encoded['iv_input']
test_encoded['iv_input'] = temp

In [20]:
test_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,vaso_input,iv_input,reward,icustayid
0,0.021211,0.028594,0.036054,0.035177,0.017385,0.033671,0.061322,0.010933,0.070122,0.080753,...,0.028383,0.034694,0.069472,0.011394,0.042277,0.011151,4.0,4.0,15.0,61.0
1,0.026839,0.038263,0.049377,0.042598,0.023414,0.028162,0.041845,0.013686,0.041322,0.118021,...,0.031515,0.024888,0.072656,0.018667,0.028841,0.011225,4.0,4.0,15.0,61.0
2,0.020956,0.037757,0.040827,0.043374,0.018417,0.034716,0.038117,0.014242,0.048841,0.116788,...,0.041911,0.020986,0.075815,0.020533,0.030378,0.012508,4.0,4.0,15.0,61.0
3,0.021644,0.037319,0.040187,0.053057,0.023536,0.039801,0.031888,0.013176,0.042813,0.121452,...,0.046401,0.017948,0.079662,0.021929,0.035035,0.013749,4.0,4.0,15.0,61.0
4,0.021647,0.040819,0.041316,0.049927,0.024191,0.039771,0.031439,0.012899,0.046829,0.131231,...,0.044421,0.017218,0.085879,0.025691,0.036103,0.014266,4.0,4.0,15.0,61.0


In [21]:
train_encoded.to_csv("../../data/train_scaled_encoded.csv", index=False)
test_encoded.to_csv("../../data/test_scaled_encoded.csv", index=False)

In [22]:
losses = [0.0982328,0.0733788,0.0571428,0.050553,0.0441328,0.0260805,0.0235437,0.0196456,0.0180331,0.0156168,0.0125812,0.0113878,0.00986484,0.00885993,0.00770599,0.00676957,0.00612195,0.00518439,0.00487075,0.00416981,0.0036631,0.00327092,0.00343543,0.00270168,0.0024088,0.00226782,0.00200392,0.00189824,0.00168429,0.00187243,0.0017625,0.00154069,0.00135107,0.0012432,0.0011839,0.00129685,0.00136126,0.0011969,0.00114256,0.00106463,0.0009524,0.00104332,0.0011232,0.000881697,0.000888103,0.000939946,0.000838807,0.000850197,0.000747317,0.000744563,0.000805789,0.000728338,0.000806852,0.000704113,0.000621048,0.000706232,0.00059969,0.000720266,0.000639709,0.00062372,0.000582317,0.000591075,0.000667023,0.000581891,0.000609561,0.000541731,0.00052279,0.000566309,0.000565176,0.000566283,0.000481784,0.000539299,0.000538973,0.000514208,0.000511358,0.000493816,0.000477208,0.000538729,0.000548364,0.000470602,0.000405979,0.000468141,0.00046181,0.000453733,0.000477661,0.000453062,0.000451573,0.000419774,0.000491614,0.000395685,0.000403904,0.000451452,0.000380449,0.000404947,0.000381785,0.000418023,0.000402506,0.000427586,0.000392169]

In [23]:
steps = [1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,15000,16000,17000,18000,19000,20000,21000,22000,23000,24000,25000,26000,27000,28000,29000,30000,31000,32000,33000,34000,35000,36000,37000,38000,39000,40000,41000,42000,43000,44000,45000,46000,47000,48000,49000,50000,51000,52000,53000,54000,55000,56000,57000,58000,59000,60000,61000,62000,63000,64000,65000,66000,67000,68000,69000,70000,71000,72000,73000,74000,75000,76000,77000,78000,79000,80000,81000,82000,83000,84000,85000,86000,87000,88000,89000,90000,91000,92000,93000,94000,95000,96000,97000,98000,99000]

In [24]:
% matplotlib inline
losses = pd.DataFrame()
